In [18]:
!pip install -r requirements.txt -q

In [19]:
# import subprocess

# subprocess.run(["aws", "s3", "sync", "s3://treetracker-training-images/pilot_annotations/PlantVillage/", "local_data/"])

In [20]:
import tqdm 
import network
import utils
import os
import random
import argparse
import numpy as np

from torch.utils import data
from datasets import LeafDataset
from utils import ext_transforms as et
from metrics import StreamSegMetrics, BinarySegMetrics

import torch
import torch.nn as nn
from utils.visualizer import Visualizer

from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
from torchvision import transforms
import matplotlib.pyplot as plt

import uuid



In [ ]:
!python3 binary_leaf_main.py --model deeplabv3plus_mobilenet --custom_data_path "local_data/" --batch_size 16 --lr 0.12 --total_itrs 800 --loss_type BCE --val_interval 100  --separable_conv --s3_model_dir "s3://treetracker-models/sagemaker_training/"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
sagemaker.config INFO - Fetched defaults config from location: /home/ec2-user/SageMaker/treetracker-machine-learning/segmentation
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.Dependencies
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.InstanceType
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker.experiments.run:The run (test) under experiment (main) already exists. Loading it.
2023-12-28 23:51:34,104 sagemaker.remote_function INFO   